<a href="https://colab.research.google.com/github/Qen-byte/Satellite-imagery/blob/main/satellite_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install GEE

In [ ]:
import ee

Authenticate Google account


In [ ]:
ee.Authenticate()

 Initialize with your project ID

In [ ]:
ee.Initialize(project='poultry-satelllite')

In [ ]:
print("Connected!")

Connected!


QUICK tEST

In [ ]:
uganda = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Uganda'))

print(f"Uganda loaded: {uganda.size().getInfo()} feature")

Uganda loaded: 1 feature


Pulling First Real Satellite Data

In [ ]:
import ee
import pandas as pd

# Define Uganda boundary
uganda = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Uganda'))

# Define time period
start_date = '2026-01-01'
end_date = '2026-02-01'

print("Fetching Sentinel-2 data...")

# Get Sentinel-2 imagery - filtering clouds
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(uganda) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

# Compute NDVI
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Compute Moisture Index
moisture = sentinel2.normalizedDifference(['B8A', 'B11']).rename('Moisture')

print("Fetching Temperature data...")

# Get Land Surface Temperature
temperature = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterBounds(uganda) \
    .filterDate(start_date, end_date) \
    .select('LST_Day_1km') \
    .mean() \
    .multiply(0.02) \
    .subtract(273.15) \
    .rename('Temperature')

print("Fetching Water Body data...")

# Get Water Bodies
water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater') \
    .select('occurrence') \
    .rename('Water')

# Combine all into one image
environmental_image = ndvi \
    .addBands(moisture) \
    .addBands(temperature) \
    .addBands(water)

print("All layers loaded successfully!")
print("Ready to extract: NDVI, Moisture, Temperature, Water")

Fetching Sentinel-2 data...
Fetching Temperature data...
Fetching Water Body data...
All layers loaded successfully!
Ready to extract: NDVI, Moisture, Temperature, Water


Load Uganda Districts

In [ ]:
# Load Uganda districts
print("Loading Uganda districts...")

districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Uganda'))

print(f"Total districts loaded: {districts.size().getInfo()}")

Loading Uganda districts...
Total districts loaded: 170


 Extract Satellite Values Per District

In [ ]:
print("Extracting environmental values per district...")
print("Please wait, this may take 3-5 minutes...")

# Combine all bands
environmental_image = ndvi \
    .addBands(moisture) \
    .addBands(temperature) \
    .addBands(water)

# Extract mean values per district
district_stats = environmental_image.reduceRegions(
    collection=districts,
    reducer=ee.Reducer.mean(),
    scale=5000  # 5km scale - faster processing, good enough for district level
)

# Convert to Python list
print("Computing results...")
stats_info = district_stats.select(
    ['ADM2_NAME', 'NDVI', 'Moisture', 'Temperature', 'Water']
).getInfo()

# Build dataframe
rows = []
for feature in stats_info['features']:
    props = feature['properties']
    rows.append({
        'district': props.get('ADM2_NAME', 'Unknown'),
        'ndvi': round(props.get('NDVI', 0) or 0, 4),
        'moisture': round(props.get('Moisture', 0) or 0, 4),
        'temperature': round(props.get('Temperature', 0) or 0, 2),
        'water': round(props.get('Water', 0) or 0, 2)
    })

df = pd.DataFrame(rows)

print(f"\nSuccessfully extracted data for {len(df)} districts!")
print("\nSample of your real satellite data:")
print(df.head(10))

Extracting environmental values per district...
Please wait, this may take 3-5 minutes...
Computing results...

Successfully extracted data for 170 districts!

Sample of your real satellite data:
                            district    ndvi  moisture  temperature  water
0                           Adjumani  0.4345   -0.0174        29.15  63.54
1                               Kole  0.5162    0.0247        28.28   0.59
2                             Kwania  0.4300    0.0626        27.58  93.00
3                             Maruzi  0.5139    0.0737        27.46  89.66
4                               Oyam  0.5150    0.0264        28.04  33.13
5  Administrative unit not available -0.0679    0.0919        23.53  98.37
6                             Bwamba  0.5605    0.2458        20.35  36.67
7                            Ntoroko  0.3724    0.1103        27.32  92.01
8                            Buhweju  0.6855    0.2017        21.87   0.00
9                        Bunyaruguru  0.4230    0.1548

Clean Data and Compute Risk Scores

In [ ]:
print("Cleaning data and computing risk scores...")

# Remove invalid districts
df = df[df['district'] != 'Administrative unit not available']
df = df[df['district'] != 'Unknown']
df = df[df['ndvi'] > 0]  # Remove negative NDVI values (invalid pixels)

# Reset index
df = df.reset_index(drop=True)

def compute_risk(row):
    risk_points = 0
    diseases = []

    ndvi = row['ndvi']
    moisture = row['moisture']
    temperature = row['temperature']
    water = row['water']

    # Coccidiosis - wet and warm conditions
    if moisture > 0.25 and temperature > 26:
        risk_points += 3
        diseases.append("Coccidiosis")

    # Newcastle Disease - dense vegetation and moisture
    if ndvi > 0.55 and moisture > 0.18:
        risk_points += 2
        diseases.append("Newcastle Disease")

    # Fowl Typhoid - high water presence
    if water > 60:
        risk_points += 2
        diseases.append("Fowl Typhoid")

    # Gumboro - moderate moisture and temperature
    if moisture > 0.15 and 24 < temperature < 30:
        risk_points += 1
        diseases.append("Gumboro")

    # Heat stress
    if temperature > 30:
        risk_points += 1
        diseases.append("Heat Stress")

    # Assign risk level
    if risk_points >= 5:
        level = "HIGH"
        color = "red"
    elif risk_points >= 3:
        level = "MEDIUM"
        color = "orange"
    else:
        level = "LOW"
        color = "green"

    return pd.Series({
        'risk_level': level,
        'risk_color': color,
        'risk_score': risk_points,
        'diseases_flagged': ', '.join(diseases) if diseases else 'None detected'
    })

# Apply risk scoring
risk_results = df.apply(compute_risk, axis=1)
df = pd.concat([df, risk_results], axis=1)

# Summary
high = len(df[df['risk_level'] == 'HIGH'])
medium = len(df[df['risk_level'] == 'MEDIUM'])
low = len(df[df['risk_level'] == 'LOW'])

print(f"Total districts analysed: {len(df)}")
print(f"HIGH risk districts:   {high}")
print(f"MEDIUM risk districts: {medium}")
print(f"LOW risk districts:    {low}")
print("\nSample risk scores:")
print(df[['district', 'risk_level', 'risk_score', 'diseases_flagged']].head(15))

Cleaning data and computing risk scores...
Total districts analysed: 153
HIGH risk districts:   1
MEDIUM risk districts: 15
LOW risk districts:    137

Sample risk scores:
             district risk_level  risk_score   diseases_flagged
0            Adjumani        LOW           2       Fowl Typhoid
1                Kole        LOW           0      None detected
2              Kwania        LOW           2       Fowl Typhoid
3              Maruzi        LOW           2       Fowl Typhoid
4                Oyam        LOW           0      None detected
5              Bwamba        LOW           2  Newcastle Disease
6             Ntoroko        LOW           2       Fowl Typhoid
7             Buhweju        LOW           2  Newcastle Disease
8         Bunyaruguru        LOW           2       Fowl Typhoid
9               Igara        LOW           2  Newcastle Disease
10            Ruhinda        LOW           2  Newcastle Disease
11             Sheema        LOW           0      None detec

 Build Heatmap

In [ ]:
# Install folium if not available
!pip install folium -q

import folium
import json

print("Building your heatmap...")

# Create base map centered on Uganda
m = folium.Map(
    location=[1.3733, 32.2903],
    zoom_start=7,
    tiles='CartoDB positron'
)

# Color mapping
def get_color(risk_level):
    if risk_level == 'HIGH':
        return 'red'
    elif risk_level == 'MEDIUM':
        return 'orange'
    else:
        return 'green'

# Add a marker for each district
for _, row in df.iterrows():
    # We will use district centroids from GEE
    folium.CircleMarker(
        location=[1.3733, 32.2903],  # placeholder - we fix in next step
        radius=10,
        color=get_color(row['risk_level']),
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"""
            <div style='font-family: Arial; width: 200px'>
                <h4 style='margin:0; color:#333'>{row['district']}</h4>
                <hr style='margin:5px 0'>
                <b>Risk Level:</b>
                <span style='color:{"red" if row["risk_level"]=="HIGH"
                    else "orange" if row["risk_level"]=="MEDIUM"
                    else "green"}'><b>{row['risk_level']}</b></span><br>
                <b>Risk Score:</b> {row['risk_score']}<br>
                <b>Diseases:</b> {row['diseases_flagged']}<br>
                <hr style='margin:5px 0'>
                <b>NDVI:</b> {row['ndvi']}<br>
                <b>Moisture:</b> {row['moisture']}<br>
                <b>Temperature:</b> {row['temperature']}°C<br>
                <b>Water Presence:</b> {row['water']}%
            </div>
            """,
            max_width=250
        )
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
     background-color: white; padding: 15px; border-radius: 10px;
     border: 2px solid grey; font-family: Arial">
    <h4 style="margin:0 0 10px 0">Poultry Disease Risk</h4>
    <div><span style="color:red">●</span> HIGH Risk</div>
    <div><span style="color:orange">●</span> MEDIUM Risk</div>
    <div><span style="color:green">●</span> LOW Risk</div>
    <hr>
    <small>Based on satellite data<br>Updated: 2026</small>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

print("Base map created!")
print("Now fetching real district coordinates...")

Building your heatmap...
Base map created!
Now fetching real district coordinates...


Get Real District Coordinates and Display Map

In [ ]:
# Get real centroids for each district from GEE
print("Fetching district centroids from GEE...")

district_coords = {}

# Get centroid for each district
for district_name in df['district'].tolist():
    try:
        district_feature = districts.filter(
            ee.Filter.eq('ADM2_NAME', district_name)
        ).first()

        centroid = district_feature.geometry().centroid()
        coords = centroid.coordinates().getInfo()
        district_coords[district_name] = [coords[1], coords[0]]  # lat, lon

    except Exception as e:
        pass

print(f"Coordinates fetched for {len(district_coords)} districts")

# Now rebuild map with real coordinates
m2 = folium.Map(
    location=[1.3733, 32.2903],
    zoom_start=7,
    tiles='CartoDB positron'
)

# Add real markers
for _, row in df.iterrows():
    district_name = row['district']

    if district_name in district_coords:
        coords = district_coords[district_name]

        folium.CircleMarker(
            location=coords,
            radius=12,
            color=get_color(row['risk_level']),
            fill=True,
            fill_color=get_color(row['risk_level']),
            fill_opacity=0.7,
            popup=folium.Popup(
                f"""
                <div style='font-family: Arial; width: 200px'>
                    <h4 style='margin:0; color:#333'>{district_name}</h4>
                    <hr style='margin:5px 0'>
                    <b>Risk Level:</b>
                    <span style='color:{"red" if row["risk_level"]=="HIGH"
                        else "orange" if row["risk_level"]=="MEDIUM"
                        else "green"}'><b>{row['risk_level']}</b></span><br>
                    <b>Risk Score:</b> {row['risk_score']}/7<br>
                    <b>Watch For:</b> {row['diseases_flagged']}<br>
                    <hr style='margin:5px 0'>
                    <small>
                    🌿 Vegetation: {row['ndvi']}<br>
                    💧 Moisture: {row['moisture']}<br>
                    🌡️ Temperature: {row['temperature']}°C<br>
                    🌊 Water: {row['water']}%
                    </small>
                </div>
                """,
                max_width=250
            )
        ).add_to(m2)

# Add legend
m2.get_root().html.add_child(folium.Element(legend_html))

# Display map directly in Colab
print("Displaying your heatmap...")
display(m2)

Fetching district centroids from GEE...
Coordinates fetched for 153 districts
Displaying your heatmap...


In [16]:
!pip install plotly -q

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

print("Building your dashboard...")

# ── 1. DATA PREPARATION ──────────────────────────────────────

# Risk distribution counts
risk_counts = df['risk_level'].value_counts()

# Top 5 highest risk districts
top5 = df.nlargest(5, 'risk_score')[['district', 'risk_score', 'risk_level']]

# Disease frequency - count how many districts flagged each disease
disease_counts = {}
diseases_list = [
    'Coccidiosis',
    'Newcastle Disease',
    'Fowl Typhoid',
    'Gumboro',
    'Heat Stress'
]

for disease in diseases_list:
    count = df['diseases_flagged'].str.contains(disease, na=False).sum()
    disease_counts[disease] = count

disease_df = pd.DataFrame(
    list(disease_counts.items()),
    columns=['Disease', 'Districts_Flagged']
).sort_values('Districts_Flagged', ascending=True)

# ── 2. CREATE DASHBOARD LAYOUT ───────────────────────────────

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'National Risk Distribution',
        'Top 5 Highest Risk Districts',
        'Disease Frequency Across Uganda',
        ''
    ),
    specs=[
        [{"type": "pie"}, {"type": "bar"}],
        [{"type": "bar", "colspan": 2}, None]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# ── 3. PIE CHART - Risk Distribution ─────────────────────────

colors_pie = []
for label in risk_counts.index:
    if label == 'HIGH':
        colors_pie.append('#e74c3c')
    elif label == 'MEDIUM':
        colors_pie.append('#f39c12')
    else:
        colors_pie.append('#2ecc71')

fig.add_trace(
    go.Pie(
        labels=risk_counts.index,
        values=risk_counts.values,
        marker=dict(colors=colors_pie),
        hole=0.4,
        textinfo='label+percent',
        hovertemplate='<b>%{label}</b><br>Districts: %{value}<br>Percentage: %{percent}<extra></extra>'
    ),
    row=1, col=1
)

# ── 4. BAR CHART - Top 5 Districts ───────────────────────────

bar_colors = []
for level in top5['risk_level']:
    if level == 'HIGH':
        bar_colors.append('#e74c3c')
    elif level == 'MEDIUM':
        bar_colors.append('#f39c12')
    else:
        bar_colors.append('#2ecc71')

fig.add_trace(
    go.Bar(
        x=top5['district'],
        y=top5['risk_score'],
        marker=dict(color=bar_colors),
        text=top5['risk_score'],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Risk Score: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# ── 5. HORIZONTAL BAR - Disease Frequency ────────────────────

disease_colors = [
    '#e74c3c', '#e67e22', '#f1c40f', '#3498db', '#9b59b6'
]

fig.add_trace(
    go.Bar(
        y=disease_df['Disease'],
        x=disease_df['Districts_Flagged'],
        orientation='h',
        marker=dict(color=disease_colors),
        text=disease_df['Districts_Flagged'],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>Districts Flagged: %{x}<extra></extra>'
    ),
    row=2, col=1
)

# ── 6. LAYOUT AND STYLING ────────────────────────────────────

fig.update_layout(
    title=dict(
        text='🐔 Uganda Poultry Disease Risk Dashboard',
        font=dict(size=22, color='#2c3e50'),
        x=0.5
    ),
    height=750,
    showlegend=False,
    paper_bgcolor='#f8f9fa',
    plot_bgcolor='#ffffff',
    font=dict(family='Arial', size=12),
    margin=dict(t=100, b=50, l=50, r=50)
)

# Clean up axes
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=True, gridcolor='#ecf0f1')

# Add a summary annotation at the bottom
fig.add_annotation(
    text=f"Data source: Sentinel-2 & MODIS Satellite Imagery | "
         f"Total Districts Monitored: {len(df)} | "
         f"Last Updated: February 2025",
    xref="paper", yref="paper",
    x=0.5, y=-0.08,
    showarrow=False,
    font=dict(size=10, color='#7f8c8d'),
    xanchor='center'
)

fig.show()
print("Dashboard displayed successfully!")

Building your dashboard...


Dashboard displayed successfully!
